<a href="https://colab.research.google.com/github/TharinsaMudalige/Neuron-Brain_Tumor_Detection_Classification_with_XAI/blob/Detection-Classficiation-CNN/Faster_R_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import Libraries

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from torchvision.ops import RoIPool
import xml.etree.ElementTree as ET
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Define Paths

In [ ]:
dataset_root = "/content/drive/MyDrive/DSGP/Preprocessed Dataset"
train_images_path = os.path.join(dataset_root, "Preprocessed Images/train")
train_annotations_path = os.path.join(dataset_root, "Annotations/train")
test_images_path = os.path.join(dataset_root, "Preprocessed Images/test")
test_annotations_path = os.path.join(dataset_root, "Annotations/test")

# Define tumor classes and labels
classes = {"No Tumor": 0, "Glioma": 1, "Meningioma": 2, "Pituitary": 3}

print("Train images path:", train_images_path)
print("Test images path:", test_images_path)
print("Train annotations path:", train_annotations_path)
print("Test annotations path:", test_annotations_path)

# Check if directories exist
print("Train images directory exists:", os.path.exists(train_images_path))
print("Test images directory exists:", os.path.exists(test_images_path))


Train images path: /content/drive/MyDrive/DSGP/Preprocessed Dataset/Preprocessed Images/train
Test images path: /content/drive/MyDrive/DSGP/Preprocessed Dataset/Preprocessed Images/test
Train annotations path: /content/drive/MyDrive/DSGP/Preprocessed Dataset/Annotations/train
Test annotations path: /content/drive/MyDrive/DSGP/Preprocessed Dataset/Annotations/test
Train images directory exists: True
Test images directory exists: True


Define Dataset Class

In [ ]:
class TumorDataset(Dataset):
    def __init__(self, img_dir, annotation_dir, transform=None):
        self.img_dir = img_dir
        self.annotation_dir = annotation_dir
        self.transform = transform
        self.imgs = [os.path.join(root, file)
                     for root, _, files in os.walk(img_dir)
                     for file in files if file.endswith(('.png', '.jpg'))]

    def __len__(self):
        return len(self.imgs)

    def __getitem__(self, idx):
        img_path = self.imgs[idx]
        annotation_path = img_path.replace(self.img_dir, self.annotation_dir).replace('.png', '.xml').replace('.jpg', '.xml')

        image = cv2.imread(img_path)
        if image is None:
            raise ValueError(f"Image not found: {img_path}")
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = torch.tensor(image, dtype=torch.float32).permute(2, 0, 1) / 255.0

        # Parse XML annotation
        tree = ET.parse(annotation_path)
        root = tree.getroot()
        boxes = []
        labels = []
        for obj in root.findall('object'):
            name = obj.find('name').text

            # Check if the tumor type is in the classes dictionary
            # If not, either skip the bounding box or handle it appropriately
            if name in classes:
                bbox = obj.find('bndbox')
                xmin = float(bbox.find('xmin').text)
                ymin = float(bbox.find('ymin').text)
                xmax = float(bbox.find('xmax').text)
                ymax = float(bbox.find('ymax').text)

                boxes.append([xmin, ymin, xmax, ymax])
                labels.append(classes[name])
            else:
                print(f"Warning: Unknown tumor type '{name}' found in annotation. Skipping this bounding box.")
                # Alternatively, you could raise an error or assign a default label

        boxes = torch.tensor(boxes, dtype=torch.float32)
        labels = torch.tensor(labels, dtype=torch.int64)

        target = {'boxes': boxes, 'labels': labels}
        return image, target

Load Train and Test Datasets

In [ ]:
train_dataset = TumorDataset(train_images_path, train_annotations_path)
test_dataset = TumorDataset(test_images_path, test_annotations_path)

train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False)

Define the Model

In [ ]:
class CustomBackbone(nn.Module):
    def __init__(self):
        super(CustomBackbone, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.pool(x)
        x = self.relu(self.conv2(x))
        x = self.pool(x)
        x = self.relu(self.conv3(x))
        x = self.pool(x)
        return x

class CustomFasterRCNN(nn.Module):
    def __init__(self, num_classes):
        super(CustomFasterRCNN, self).__init__()
        self.backbone = CustomBackbone()
        self.roi_pool = RoIPool(output_size=(7, 7), spatial_scale=1.0 / 16.0)
        self.fc1 = nn.Linear(256 * 7 * 7, 1024)
        self.fc2 = nn.Linear(1024, 512)
        self.cls_layer = nn.Linear(512, num_classes)
        self.reg_layer = nn.Linear(512, num_classes * 4)

    def forward(self, x, rois):
        features = self.backbone(x)
        pooled = self.roi_pool(features, rois)
        pooled = pooled.view(pooled.size(0), -1)
        pooled = F.relu(self.fc1(pooled))
        pooled = F.relu(self.fc2(pooled))
        classification = self.cls_layer(pooled)
        bbox_regression = self.reg_layer(pooled)
        return classification, bbox_regression

Train the Model

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = CustomFasterRCNN(num_classes=len(classes)).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.0001)
criterion = nn.CrossEntropyLoss()

num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, targets in train_loader:
        images = images.to(device)
        optimizer.zero_grad()
        rois = torch.rand((len(images), 5)).to(device)
        cls_pred, bbox_pred = model(images, rois)
        loss = criterion(cls_pred, targets['labels'])
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch + 1}, Loss: {running_loss:.4f}")

RuntimeError: stack expects each tensor to be equal size, but got [0] at entry 0 and [1, 4] at entry 2